In [13]:
import numpy as np
import os
import pandas as pd #data handling library
import sklearn as sk

def get_RPKM_by_file(file_name, data_fields):
    df = pd.read_table(file_name, usecols=data_fields)
    df = df.T # transpose
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df = df.rename(columns = new_header.T) #set the header row as the df header
    return df


def get_RPKM_by_filelist(list_fname):
    RPKM_list = []
    data_fields = ('gene','RPKM')
    with open(list_fname, 'r') as f:
        RPKM_list.extend(map(lambda data_file: get_RPKM_by_file(data_file, data_fields), f.readlines()))
    return RPKM_list

def get_RPKM_by_dir(data_dir):
    RPKM_list = []
    data_fields = ('gene','RPKM')
    for dirName, subdirList, fileList in os.walk(data_dir):
        RPKM_list.extend(map(lambda f: get_RPKM_by_file(os.path.join(dirName, f), data_fields), fileList))
    return RPKM_list

#data_dir = './Data'
#RPKM_list = get_RPKM_by_dir(data_dir)
RPKM_list = get_RPKM_by_filelist('files.index')

In [20]:
def get_labels(filename):
    col = ('sample_type')
    df = pd.read_table(filename)
    return df
print get_labels("metadata.csv").head(n=5)

['#case_id,age_at_diagnosis,vital_status,days_to_death,gender,sample_type,race,ethnicity,platform']


,"#case_id,age_at_diagnosis,vital_status,days_to_death,gender,sample_type,race,ethnicity,platform"
0,"TCGA-GM-A2DC,57,Alive,,FEMALE,Primary Tumor,WH..."
1,"TCGA-GM-A2D9,69,Dead,1812,FEMALE,Primary Tumor..."
2,"TCGA-GM-A2DB,62,Alive,,FEMALE,Primary Tumor,WH..."
3,"TCGA-AC-A2QH,58,Alive,,FEMALE,Primary Tumor,WH..."
4,"TCGA-AR-A2LO,46,Alive,,FEMALE,Primary Tumor,WH..."


In [ ]:

#def get_RPKM_by_file(files):
#    for file_name in files:
#        df = pd.read_table(file_name, usecols=data_fields)
#        df = df.T # tanspose
#        new_header = df.iloc[0] #grab the first row for the header
#        df = df[1:] #take the data less the header row
#        df = df.rename(columns = new_header.T) #set the header row as the df header
#        RPKM_list.append(df)
#    return RPKM_list



#flattened_data = get_RPKM_by_file(file_list)




In [ ]:
def merge_data(data_list):
    df = pd.concat(data_list, axis=0, join='inner', join_axes=None, ignore_index=False,
        keys=None, levels=None, names=None, verify_integrity=False)
    return df

X = merge_data(RPKM_list)
Y = np.ravel(map(round, np.random.rand(len(X), 1)))

print X

In [ ]:
# Impute missing data, assuming X is all numeric values. Y can be strings.
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X = imp.fit_transform(X)

imp = Imputer(missing_values='NaN', strategy='most_frequent', axis=0)
Y = imp.fit_transform(Y)

In [ ]:
# Encode classes as numbers
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
nonnumeric_columns_X = []
nonnumeric_columns_Y = []
if nonnumeric_columns_X:
    for feature in nonnumeric_columns_X:
        X[feature] = le.fit_transform(X[feature])
if nonnumeric_columns_Y:
    for feature in nonnumeric_columns_Y:
        Y[feature] = le.fit_transform(Y[feature])

In [ ]:
# Split data into train and test sets.
def split_data(X, Y, p_train):
    msk = np.random.rand(len(RPKM_list)) < p_train
    print Y[0]
    print msk
    X_train = X[msk]
    Y_train = Y[0][msk]
    return X[msk], X[~msk], Y[0][msk], Y[0][~msk]

train_X, test_X, train_Y, test_Y = split_data(X, Y, 0.6)
print train_Y
print test_Y